In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d ayuraj/asl-dataset
#downloading the dataset

In [ ]:
!unzip asl-dataset.zip -d asl_data

In [ ]:
import numpy as np  #for performing numerical
import tensorflow as tf
import matplotlib.pyplot as plt#helps in using graphs and images
import os      #helps in directing directories and files
from tensorflow.keras.preprocessing.image import ImageDataGenerator #helps in importing all files and helps in changing limited datset(augmentation)

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255.0,validation_split=0.2,
rotation_range=20,
zoom_range=0.2,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip='true') # Removed the trailing comma

In [ ]:
train_generator=train_datagen.flow_from_directory(
'/content/asl_data/asl_dataset',
target_size=(64,64),
batch_size=32,
color_mode='grayscale',
class_mode='categorical',
subset='training'
)
val_generator=train_datagen.flow_from_directory(
'/content/asl_data/asl_dataset',
target_size=(64,64),
batch_size=32,
color_mode='grayscale',
class_mode='categorical',
subset='validation'
)

In [ ]:
import os

dataset_path = '/content/asl_data/asl_dataset'
classes = sorted(os.listdir(dataset_path))
print(f"Number of classes found: {len(classes)}")
print("Classes:", classes)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

model=Sequential([
Conv2D(64,(3,3),activation='relu',input_shape=(64,64,1)),
MaxPooling2D(2,2),
Conv2D(128,(3,3),activation='relu'),
MaxPooling2D(2,2),
Flatten(),
Dense(128,activation='relu'),
Dropout(0.5),
Dense(37,activation='softmax') # Changed to 37 units to match the number of classes
])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator,validation_data=val_generator,epochs=5) # Assign the output of model.fit to history

In [ ]:
model.save('asl_model.h5')


In [ ]:
plt.plot(history.history['accuracy'],label=['train accuracy'])
plt.plot(history.history['val_accuracy'],label='val accuracy')
plt.show()
plt.axis('off')


In [ ]:
from google.colab import files
files.upload()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(64, 64), color_mode='grayscale')
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('asl_model.h5')  # Make sure this is your trained model

def predict_sign(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    print(f"Predicted Letter: {chr(predicted_class + 65)}")

In [ ]:
predict_sign('gest_5.png')
